# Semantično jedro z integracijo OpenBnB MCP strežnika

Ta zvezek prikazuje, kako uporabiti semantično jedro z dejanskim OpenBnB MCP strežnikom za iskanje resničnih nastanitev Airbnb z uporabo MCPStdioPlugin. Za dostop do LLM uporablja Azure AI Foundry. Za nastavitev okoljskih spremenljivk lahko sledite [Lekciji za nastavitev](/00-course-setup/README.md)


## Uvoz potrebnih paketov


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Ustvarjanje povezave za MCP vtičnik

Povezali se bomo s [OpenBnB MCP strežnikom](https://github.com/openbnb-org/mcp-server-airbnb) z uporabo MCPStdioPlugin. Ta strežnik omogoča iskanje po Airbnb prek paketa @openbnb/mcp-server-airbnb.


## Ustvarjanje odjemalca

V tem primeru bomo uporabili Azure AI Foundry za dostop do LLM. Prepričajte se, da so vaše okoljske spremenljivke pravilno nastavljene.


## Konfiguracija okolja

Nastavite Azure OpenAI nastavitve. Prepričajte se, da imate nastavljene naslednje spremenljivke okolja:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Razumevanje integracije OpenBnB MCP

Ta zvezek se povezuje z **pravim OpenBnB MCP strežnikom**, ki omogoča dejansko funkcionalnost iskanja na Airbnb.

### Kako deluje:

1. **MCPStdioPlugin**: Uporablja komunikacijo prek standardnega vhoda/izhoda s strežnikom MCP
2. **Pravi NPM paket**: Prenese in zažene `@openbnb/mcp-server-airbnb` prek npx
3. **Živi podatki**: Vrača dejanske podatke o nepremičninah Airbnb iz njihovih API-jev
4. **Odkritje funkcij**: Agent samodejno odkrije razpoložljive funkcije strežnika MCP

### Razpoložljive funkcije:

Strežnik OpenBnB MCP običajno omogoča:
- **search_listings** - Iskanje nepremičnin Airbnb glede na lokacijo in kriterije
- **get_listing_details** - Pridobitev podrobnih informacij o določenih nepremičninah
- **check_availability** - Preverjanje razpoložljivosti za določene datume
- **get_reviews** - Pridobitev ocen za nepremičnine
- **get_host_info** - Pridobitev informacij o gostiteljih nepremičnin

### Predpogoji:

- **Node.js** nameščen na vašem sistemu
- **Internetna povezava** za prenos paketa strežnika MCP
- **NPX** na voljo (prihaja z Node.js)

### Testiranje povezave:

Povezavo s strežnikom MCP lahko ročno testirate z zagonom:
```bash
npx -y @openbnb/mcp-server-airbnb
```

To bo preneslo in zagnalo OpenBnB MCP strežnik, na katerega se Semantic Kernel nato poveže za pridobivanje dejanskih podatkov Airbnb.


## Zagon agenta z OpenBnB MCP strežnikom

Zdaj bomo zagnali AI agenta, ki se povezuje z OpenBnB MCP strežnikom za iskanje pravih Airbnb nastanitev v Stockholmu za 2 odrasla in 1 otroka. Seznam `user_inputs` lahko poljubno spremenite, da prilagodite iskalna merila.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Povzetek  
Čestitke! Uspešno ste ustvarili AI agenta, ki se povezuje z iskanjem nastanitev v resničnem svetu prek protokola Model Context Protocol (MCP):

Uporabljene tehnologije:  
Semantic Kernel - Za gradnjo inteligentnih agentov z Azure OpenAI  
Azure AI Foundry - Za zmogljivosti LLM in zaključevanje pogovorov  
MCP (Model Context Protocol) - Za standardizirano integracijo orodij  
OpenBnB MCP Server - Za resnično funkcionalnost iskanja Airbnb  
Node.js/NPX - Za zagon zunanjega MCP strežnika  

Kaj ste se naučili:  
MCP integracija: Povezovanje Semantic Kernel agentov z zunanjimi MCP strežniki  
Dostop do podatkov v realnem času: Iskanje dejanskih Airbnb nastanitev prek živih API-jev  
Komunikacija prek protokola: Uporaba stdio komunikacije med agentom in MCP strežnikom  
Odkritje funkcij: Samodejno odkrivanje razpoložljivih funkcij iz MCP strežnikov  
Pretok odgovorov: Zajemanje in beleženje klicev funkcij v realnem času  
HTML upodabljanje: Oblikovanje odgovorov agenta s stiliziranimi tabelami in interaktivnimi prikazi  

Naslednji koraki:  
Integrirajte dodatne MCP strežnike (vreme, leti, restavracije)  
Zgradite sistem več agentov, ki združuje MCP in A2A protokole  
Ustvarite prilagojene MCP strežnike za svoje podatkovne vire  
Implementirajte trajni spomin pogovorov med sejami  
Namestite agenta v Azure Functions z orkestracijo MCP strežnika  
Dodajte uporabniško avtentikacijo in funkcionalnosti rezervacij  

Ključne prednosti MCP arhitekture:  
Standardizacija: Univerzalni protokol za povezovanje AI agentov z zunanjimi orodji  
Podatki v realnem času: Dostop do živih, posodobljenih informacij iz različnih storitev  
Razširljivost: Enostavna integracija novih podatkovnih virov in orodij  
Interoperabilnost: Deluje z različnimi AI ogrodji in platformami za agente  
Ločitev odgovornosti: Jasna razmejitev med AI logiko in dostopom do zunanjih podatkov



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni prevod s strani človeka. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
